<h1> Feature Engineering of Instacart-Market-Basket-Analysis </h1>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

from PIL import Image
from astropy.stats import circmean, circvar

from tqdm import tqdm
import sqlite3
import pickle
from scipy.linalg import svd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD 

<h1> Creating Data using Datasets </h1>

In [2]:
aisles_data = pd.read_csv("aisles.csv")
departments_data = pd.read_csv("departments.csv")
or_prod_prior = pd.read_csv("order_products__prior.csv")
or_prod_train = pd.read_csv("order_products__train.csv")
orders_data = pd.read_csv("orders.csv")
product_data = pd.read_csv("products.csv")

In [3]:
prior_orders = pd.merge(orders_data,or_prod_prior, how='inner')
prior_orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0
...,...,...,...,...,...,...,...,...,...,...
32434484,2977660,206209,prior,13,1,12,7.0,14197,5,1
32434485,2977660,206209,prior,13,1,12,7.0,38730,6,0
32434486,2977660,206209,prior,13,1,12,7.0,31477,7,0
32434487,2977660,206209,prior,13,1,12,7.0,6567,8,0


In [4]:
del or_prod_prior

In [5]:
#Provided by the Applied Ai team
#URL : https://www.kaggle.com/rinnqd/reduce-memory-usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

<h1> Cyclic-Features </h1>
<br>
<div>
    <ol>
        <li>periodic-order-dow</li>
        <li>periodic-order-hod</li>
        <li>periodic-order-dsp</li>
    </ol>

In [9]:
def transform(row, oscillation):
    """
    This feature will convert row with oscillation to periodic distribution.
    parameter : row(array) , oscillation(scalar)
    return    : periodic row(array)
    """
    return row.multiply(2 * np.pi / oscillation).sub(np.pi).astype('float16')

In [8]:
print("Creating Cyclic Features...")
cyclic_features = pd.DataFrame()

cyclic_features["user_id"] = prior_orders.user_id.copy()
cyclic_features["product_id"] = prior_orders.product_id.copy()
cyclic_features["periodic-order-dow"] = transform(prior_orders.order_dow , 7)
print("Created periodic-order-dow")
cyclic_features["periodic-order-hod"] = transform(prior_orders.order_hour_of_day , 24)
print("Created periodic-order-hod")
cyclic_features["periodic-order-dsp"] = transform(prior_orders.days_since_prior_order , 30)
print("Created periodic-order-dsp")

cyclic_features = reduce_mem_usage(cyclic_features)
cyclic_features.info()

Creating Cyclic Features...
Created periodic-order-dow
Created periodic-order-hod
Created periodic-order-dsp
Memory usage after optimization is: 680.50 MB
Decreased by 26.7%
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 5 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             int32  
 1   product_id          int32  
 2   periodic-order-dow  float16
 3   periodic-order-hod  float16
 4   periodic-order-dsp  float16
dtypes: float16(3), int32(2)
memory usage: 680.5 MB


In [9]:
#Saving the file
with open("cyclic_features.pickle" , "wb") as file:
    pickle.dump(cyclic_features,file)

In [10]:
#Cleaning 
del cyclic_features

<h1> User-Features </h1>
<br>
<ol>
        <li>max_orders</li>	
        <li>total_items</li>	
        <li>orders_ratio</li>		
        <li>unique_products</li>
        <li>num_reorders</li>
        <li>reorder_mean</li>	
        <li>reorder_std</li>		
        <li>user_dow_mean</li>	
        <li>user_dow_var</li>	
        <li>user_dow_std</li>	
        <li>user_hod_mean</li>	
        <li>user_hod_var</li>	
        <li>user_hod_std</li>	
        <li>user_dsp_mean</li>	
        <li>user_dsp_var</li>
        <li>user_dsp_std</li>
   </ol>

In [19]:
#max_orders:Max orders ordered by users
max_orders = prior_orders.groupby(by='user_id')['order_number'].max().to_frame('max_orders')
max_orders = max_orders.reset_index()
max_orders

,user_id,max_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4
...,...,...
206204,206205,3
206205,206206,67
206206,206207,16
206207,206208,49


In [20]:
#total_items:Total items bought by the user
total_items = prior_orders.groupby(by="user_id")["product_id"].count().to_frame("total_items").reset_index()
total_items

,user_id,total_items
0,1,59
1,2,195
2,3,88
3,4,18
4,5,37
...,...,...
206204,206205,32
206205,206206,285
206206,206207,223
206207,206208,677


In [21]:
#orders_ratio = max_orders_by_total_items
temp_data = pd.merge(max_orders , total_items , on="user_id" , how="left")
temp_data["orders_ratio"] = temp_data["max_orders"] / temp_data["total_items"]
orders_ratio = temp_data[["user_id" , "orders_ratio"]]
orders_ratio

,user_id,orders_ratio
0,1,0.169492
1,2,0.071795
2,3,0.136364
3,4,0.277778
4,5,0.108108
...,...,...
206204,206205,0.093750
206205,206206,0.235088
206206,206207,0.071749
206207,206208,0.072378


In [22]:
del temp_data

In [23]:
#unique_products:No of unique products by the user
unique_products = prior_orders.groupby('user_id')['product_id'].nunique().to_frame("unique_products").reset_index()
unique_products

,user_id,unique_products
0,1,18
1,2,102
2,3,33
3,4,17
4,5,23
...,...,...
206204,206205,24
206205,206206,150
206206,206207,92
206207,206208,198


In [24]:
#num_reorders : num of reorders by the users
num_reorders = prior_orders.groupby(by="user_id")["reordered"].sum().to_frame("num_reorders").reset_index()
num_reorders

,user_id,num_reorders
0,1,41
1,2,93
2,3,55
3,4,1
4,5,14
...,...,...
206204,206205,8
206205,206206,135
206206,206207,131
206207,206208,479


In [25]:
#reordered_mean: mean of reordering of the user
reordered_mean = prior_orders.groupby(by="user_id")["reordered"].mean().to_frame("reordered_mean").reset_index()
reordered_mean

,user_id,reordered_mean
0,1,0.694915
1,2,0.476923
2,3,0.625000
3,4,0.055556
4,5,0.378378
...,...,...
206204,206205,0.250000
206205,206206,0.473684
206206,206207,0.587444
206207,206208,0.707533


In [26]:
#reordered_std : Standard Deviation of reorders of the user
reordered_std = prior_orders.groupby(by="user_id")["reordered"].std().to_frame("reordered_std").reset_index()
reordered_std

,user_id,reordered_std
0,1,0.464396
1,2,0.500753
2,3,0.486897
3,4,0.235702
4,5,0.491672
...,...,...
206204,206205,0.439941
206205,206206,0.500185
206206,206207,0.493402
206207,206208,0.455232


In [31]:
#user_dow_mean : mean of day of the week of the user
user_dow_mean = prior_orders.groupby(by="user_id")["order_dow"].mean().to_frame("user_dow_mean").reset_index()
user_dow_mean

,user_id,user_dow_mean
0,1,2.644068
1,2,2.005128
2,3,1.011364
3,4,4.722222
4,5,1.621622
...,...,...
206204,206205,3.718750
206205,206206,2.312281
206206,206207,2.896861
206207,206208,2.760709


In [32]:
#user_dow_var : var of day of the week of the user
user_dow_var = prior_orders.groupby(by="user_id")["order_dow"].var().to_frame("user_dow_var").reset_index()
user_dow_var

,user_id,user_dow_var
0,1,1.578025
1,2,0.943273
2,3,1.551594
3,4,0.683007
4,5,1.630631
...,...,...
206204,206205,1.176411
206205,206206,3.722560
206206,206207,4.210035
206207,206208,3.007744


In [33]:
#user_dow_std : std of day of the week of the user
user_dow_std = prior_orders.groupby(by="user_id")["order_dow"].std().to_frame("user_dow_std").reset_index()
user_dow_std

,user_id,user_dow_std
0,1,1.256194
1,2,0.971222
2,3,1.245630
3,4,0.826442
4,5,1.276961
...,...,...
206204,206205,1.084625
206205,206206,1.929394
206206,206207,2.051837
206207,206208,1.734285


In [34]:
#user_hod_mean : mean of the user hour of the day
user_hod_mean = prior_orders.groupby(by="user_id")["order_hour_of_day"].mean().to_frame("user_hod_mean").reset_index()
user_hod_mean

,user_id,user_hod_mean
0,1,10.542373
1,2,10.441026
2,3,16.352273
3,4,13.111111
4,5,15.729730
...,...,...
206204,206205,13.625000
206205,206206,16.796491
206206,206207,13.130045
206207,206208,13.968981


In [35]:
#user_hod_var : var of the user hour of the day
user_hod_var = prior_orders.groupby(by="user_id")["order_hour_of_day"].var().to_frame("user_hod_var").reset_index()
user_hod_var

,user_id,user_hod_var
0,1,12.252484
1,2,2.722020
2,3,2.115857
3,4,3.045752
4,5,6.702703
...,...,...
206204,206205,3.209677
206205,206206,5.211959
206206,206207,20.248778
206207,206208,14.462054


In [36]:
#user_hod_std : std of the user hour of the day
user_hod_std =  prior_orders.groupby(by="user_id")["order_hour_of_day"].std().to_frame("user_hod_std").reset_index()
user_hod_std

,user_id,user_hod_std
0,1,3.500355
1,2,1.649854
2,3,1.454599
3,4,1.745208
4,5,2.588958
...,...,...
206204,206205,1.791557
206205,206206,2.282972
206206,206207,4.499864
206207,206208,3.802901


In [37]:
#user_dsp_mean : mean of user day since prior order
user_dsp_mean = prior_orders.groupby(by="user_id")["days_since_prior_order"].mean().to_frame("user_dsp_mean").reset_index()
user_dsp_mean

,user_id,user_dsp_mean
0,1,20.259259
1,2,15.967033
2,3,11.487179
3,4,15.357143
4,5,14.500000
...,...,...
206204,206205,20.666667
206205,206206,4.042705
206206,206207,14.879397
206207,206208,7.442105


In [38]:
#user_dsp_var : var of user day since prior order
user_dsp_var = prior_orders.groupby(by="user_id")["days_since_prior_order"].var().to_frame("user_dsp_var").reset_index()
user_dsp_var

,user_id,user_dsp_var
0,1,86.573026
1,2,83.170178
2,3,23.707626
3,4,73.631868
4,5,18.180000
...,...,...
206204,206205,106.666667
206205,206206,11.926741
206206,206207,127.551038
206207,206208,16.180755


In [39]:
#user_dsp_std : var of user day since prior order
user_dsp_std = prior_orders.groupby(by="user_id")["days_since_prior_order"].std().to_frame("user_dsp_std").reset_index()
user_dsp_std

,user_id,user_dsp_std
0,1,9.304463
1,2,9.119769
2,3,4.869048
3,4,8.580901
4,5,4.263801
...,...,...
206204,206205,10.327956
206205,206206,3.453511
206206,206207,11.293850
206207,206208,4.022531


<h1> Concatenating All Users Features </h1>

In [48]:
users = pd.DataFrame()
users["user_id"] = max_orders.user_id.copy()
instances = [max_orders , total_items , orders_ratio , unique_products , num_reorders , reordered_mean , reordered_std , user_dow_mean ,user_dow_var ,user_dow_std,user_hod_mean,user_hod_var,user_hod_std,user_dsp_mean,user_dsp_var,user_dsp_std]
print("Merging all users data frame.....")
for instance in tqdm(instances):
    users = pd.merge(users , instance , on="user_id" , how="left")
print("Merging Done..")
print(users.info())


 12%|█▎        | 2/16 [00:00<00:00, 17.54it/s]

Merging all users data frame.....


100%|██████████| 16/16 [00:01<00:00, 15.46it/s]

Merging Done..
<class 'pandas.core.frame.DataFrame'>
Int64Index: 206209 entries, 0 to 206208
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   user_id          206209 non-null  int64  
 1   max_orders       206209 non-null  int64  
 2   total_items      206209 non-null  int64  
 3   orders_ratio     206209 non-null  float64
 4   unique_products  206209 non-null  int64  
 5   num_reorders     206209 non-null  int64  
 6   reordered_mean   206209 non-null  float64
 7   reordered_std    206209 non-null  float64
 8   user_dow_mean    206209 non-null  float64
 9   user_dow_var     206209 non-null  float64
 10  user_dow_std     206209 non-null  float64
 11  user_hod_mean    206209 non-null  float64
 12  user_hod_var     206209 non-null  float64
 13  user_hod_std     206209 non-null  float64
 14  user_dsp_mean    206209 non-null  float64
 15  user_dsp_var     206209 non-null  float64
 16  user_dsp_std     206209

In [49]:
users = reduce_mem_usage(users)

with open("users.pickle" , "wb") as file:
    pickle.dump(users , file)


Memory usage after optimization is: 8.46 MB
Decreased by 70.1%


In [51]:
#Cleaning
print("Cleaning....")
for instance in instances:
    del instance
print("Done")

Cleaning....
Done


In [52]:
del users

<h1> Product-features </h1>
<br>
<div>
    <ol>
        <li>prod_sum_on</li>
        <li>prod_sum_dow</li>
        <li>prod_sum_hod</li>
        <li>prod_sum_dsp</li>
        <li>prod_sum_atc</li>
        <li>prod_sum_reordered</li>
        <li>prod_num</li>	
        <li>prod_unique_users</li>	
        <li>prod_reorder_mean</li>	
        <li>prod_order_hour_of_day_mean</li>	
        <li>prod_order_hour_of_day_var</li>	
        <li>prod_order_dow_mean</li>	
        <li>prod_order_dow_var</li>
    </ol>
</div>


In [6]:
instances = []

In [7]:
prior_orders.groupby(["product_id"]).sum()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered
product_id,,,,,,,,
1,3174449206,189108251,35924,5142,24517,18299.0,10745,1136
2,154118818,7939202,2310,263,1195,912.0,890,12
3,499419645,29353658,4804,758,3353,2747.0,1777,203
4,559017713,34731481,2970,883,4512,4259.0,3128,147
5,26839133,2139938,134,41,160,174.0,97,9
...,...,...,...,...,...,...,...,...
49684,19268304,1055586,43,38,103,95.0,39,1
49685,83686768,5363207,659,135,670,582.0,469,6
49686,195016581,15340838,2318,346,1574,963.0,900,84


In [8]:
#prod_sum_on : product sum on order_number
prod_sum_on = prior_orders.groupby(["product_id"])["order_number"].sum().to_frame("prod_sum_on").reset_index()
prod_sum_on

,product_id,prod_sum_on
0,1,35924
1,2,2310
2,3,4804
3,4,2970
4,5,134
...,...,...
49672,49684,43
49673,49685,659
49674,49686,2318
49675,49687,180


In [9]:
instances.append(prod_sum_on)

In [10]:
#prod_sum_dow : product sum on day of thhe week
prod_sum_dow = prior_orders.groupby(["product_id"])["order_dow"].sum().to_frame("prod_sum_dow").reset_index()
prod_sum_dow

,product_id,prod_sum_dow
0,1,5142
1,2,263
2,3,758
3,4,883
4,5,41
...,...,...
49672,49684,38
49673,49685,135
49674,49686,346
49675,49687,32


In [11]:
instances.append(prod_sum_dow)

In [12]:
#prod_sum_hod : product sum on hour of day
prod_sum_hod = prior_orders.groupby(["product_id"])["order_hour_of_day"].sum().to_frame("prod_sum_hod").reset_index()
prod_sum_hod

,product_id,prod_sum_hod
0,1,24517
1,2,1195
2,3,3353
3,4,4512
4,5,160
...,...,...
49672,49684,103
49673,49685,670
49674,49686,1574
49675,49687,162


In [13]:
instances.append(prod_sum_hod)

In [14]:
#prod_sum_dsp : product sum on days since prior order
prod_sum_dsp = prior_orders.groupby(["product_id"])["days_since_prior_order"].sum().to_frame("prod_sum_dsp").reset_index()
prod_sum_dsp

,product_id,prod_sum_dsp
0,1,18299.0
1,2,912.0
2,3,2747.0
3,4,4259.0
4,5,174.0
...,...,...
49672,49684,95.0
49673,49685,582.0
49674,49686,963.0
49675,49687,202.0


In [15]:
instances.append(prod_sum_dsp)

In [16]:
#add_to_cart_order	reordered
#prod_sum_atc : product sum with add to cart order
prod_sum_atc = prior_orders.groupby(["product_id"])["add_to_cart_order"].sum().to_frame("prod_sum_atc").reset_index()
prod_sum_atc

,product_id,prod_sum_atc
0,1,10745
1,2,890
2,3,1777
3,4,3128
4,5,97
...,...,...
49672,49684,39
49673,49685,469
49674,49686,900
49675,49687,98


In [17]:
instances.append(prod_sum_atc)

In [18]:
#prod_sum_reordered : product sum with reordered 
prod_sum_reordered = prior_orders.groupby(["product_id"])["reordered"].sum().to_frame("prod_sum_reordered").reset_index()
prod_sum_reordered

,product_id,prod_sum_reordered
0,1,1136
1,2,12
2,3,203
3,4,147
4,5,9
...,...,...
49672,49684,1
49673,49685,6
49674,49686,84
49675,49687,6


In [19]:
instances.append(prod_sum_reordered)

In [20]:
#prod_num : number of time product has been bought
prod_num = prior_orders.groupby(["product_id"])["order_id"].count().to_frame("prod_num").reset_index()
prod_num

,product_id,prod_num
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15
...,...,...
49672,49684,9
49673,49685,49
49674,49686,120
49675,49687,13


In [21]:
instances.append(prod_num)

In [22]:
# prod_unique_users : unique users for the products
prod_unique_users = prior_orders.groupby(by="product_id")["user_id"].nunique().to_frame("prod_unique_users").reset_index()
prod_unique_users

,product_id,prod_unique_users
0,1,716
1,2,78
2,3,74
3,4,182
4,5,6
...,...,...
49672,49684,8
49673,49685,43
49674,49686,36
49675,49687,7


In [23]:
instances.append(prod_unique_users)

In [24]:
#prod_reorder_mean : product reorder mean
prod_reorder_mean = prior_orders.groupby(by="product_id")["reordered"].mean().to_frame("prod_reorder_mean").reset_index()
prod_reorder_mean

,product_id,prod_reorder_mean
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000
...,...,...
49672,49684,0.111111
49673,49685,0.122449
49674,49686,0.700000
49675,49687,0.461538


In [25]:
instances.append(prod_reorder_mean)

In [26]:
# prod_order_hour_of_day_mean : products hour of the day mean
prod_order_hour_of_day_mean = prior_orders.groupby(by="product_id")["order_hour_of_day"].mean().to_frame("prod_order_hour_of_day_mean").reset_index()
prod_order_hour_of_day_mean

,product_id,prod_order_hour_of_day_mean
0,1,13.238121
1,2,13.277778
2,3,12.104693
3,4,13.714286
4,5,10.666667
...,...,...
49672,49684,11.444444
49673,49685,13.673469
49674,49686,13.116667
49675,49687,12.461538


In [27]:
instances.append(prod_order_hour_of_day_mean)

In [28]:
# prod_order_hour_of_day_var : products hour of the day var
prod_order_hour_of_day_var = prior_orders.groupby(by="product_id")["order_hour_of_day"].var().to_frame("prod_order_hour_of_day_var").reset_index()
prod_order_hour_of_day_var

,product_id,prod_order_hour_of_day_var
0,1,18.104802
1,2,15.528714
2,3,20.383927
3,4,19.107143
4,5,20.666667
...,...,...
49672,49684,8.777778
49673,49685,18.391156
49674,49686,23.683754
49675,49687,12.102564


In [29]:
instances.append(prod_order_hour_of_day_var)

In [30]:
#prod_order_dow_mean : product day of the week mean
prod_order_dow_mean = prior_orders.groupby(by="product_id")["order_dow"].mean().to_frame("prod_order_dow_mean").reset_index()
prod_order_dow_mean

,product_id,prod_order_dow_mean
0,1,2.776458
1,2,2.922222
2,3,2.736462
3,4,2.683891
4,5,2.733333
...,...,...
49672,49684,4.222222
49673,49685,2.755102
49674,49686,2.883333
49675,49687,2.461538


In [31]:
instances.append(prod_order_dow_mean)

In [32]:
#prod_order_dow_var : product day of the week var
prod_order_dow_var = prior_orders.groupby(by="product_id")["order_dow"].var().to_frame("prod_order_dow_var").reset_index()
prod_order_dow_var

,product_id,prod_order_dow_var
0,1,3.333038
1,2,4.634332
2,3,4.861456
3,4,4.649770
4,5,5.209524
...,...,...
49672,49684,3.944444
49673,49685,3.605442
49674,49686,4.877031
49675,49687,2.935897


In [33]:
instances.append(prod_order_dow_var)

In [34]:
print(len(instances))

13


<h1> Concatenating all product features </h1>

In [35]:
products = pd.DataFrame()
products["product_id"] = prod_order_dow_var.product_id.copy()
#instances = [max_orders , total_items , orders_ratio , unique_products , num_reorders , reordered_mean , reordered_std , user_dow_mean ,user_dow_var ,user_dow_std,user_hod_mean,user_hod_var,user_hod_std,user_dsp_mean,user_dsp_var,user_dsp_std]
print("Merging all users data frame.....")
for instance in tqdm(instances):
    products = pd.merge(products , instance , on="product_id" , how="left")
print("Merging Done..")
print(products.info())

100%|██████████| 13/13 [00:00<00:00, 71.78it/s]

Merging all users data frame.....
Merging Done..
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49677 entries, 0 to 49676
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_id                   49677 non-null  int64  
 1   prod_sum_on                  49677 non-null  int64  
 2   prod_sum_dow                 49677 non-null  int64  
 3   prod_sum_hod                 49677 non-null  int64  
 4   prod_sum_dsp                 49677 non-null  float64
 5   prod_sum_atc                 49677 non-null  int64  
 6   prod_sum_reordered           49677 non-null  int64  
 7   prod_num                     49677 non-null  int64  
 8   prod_unique_users            49677 non-null  int64  
 9   prod_reorder_mean            49677 non-null  float64
 10  prod_order_hour_of_day_mean  49677 non-null  float64
 11  prod_order_hour_of_day_var   49546 non-null  float64
 12  prod_order_dow_mean      

In [36]:
products.fillna(0 , inplace = True)

products = reduce_mem_usage(products)

with open("products.pickle" , "wb") as file:
    pickle.dump(products,file)


Memory usage after optimization is: 2.56 MB
Decreased by 55.0%


In [37]:
#Cleaning
print("Cleaning....")
for instance in instances:
    del instance
print("Done")

Cleaning....
Done


In [38]:
del products

<h1> User-Poduct Features </h1>
<br>
<ol>
    <li>up_unique_dow</li>
    <li>up_unique_hod</li>
    <li>up_unique_atc</li>
    <li>up_unique_reorder</li>
    <li>up_cumcount_on</li>
    <li>up_cumcount_dow</li>
    <li>up_cumcount_hod</li>
    <li>up_cumcount_dsp</li>
    <li>up_cumcount_atc</li>
    <li>up_orders_cumcount </li>
    <li>up_orders_num</li>	
    <li>up_orders_since_previous</li>		
    <li>up_reordered</li>	
    <li>up_order_ratio</li>		
    <li>up_order_dow_score</li>	
    <li>up_order_hour_of_day_score</li>
    <li>up_last_five</li>
    <li>up_last_five_ratio</li>
 </ol>

In [9]:
instances = []

In [8]:
prior_orders.groupby(["user_id","product_id"]).nunique()

order_id  eval_set  order_number  order_dow  \
user_id product_id                                                
1       196               10         1            10          4   
        10258              9         1             9          4   
        10326              1         1             1          1   
        12427             10         1            10          4   
        13032              3         1             3          3   
...                      ...       ...           ...        ...   
206209  43961              3         1             3          2   
        44325              1         1             1          1   
        48370              1         1             1          1   
        48697              1         1             1          1   
        48742              2         1             2          2   

                    order_hour_of_day  days_since_prior_order  \
user_id product_id                                              
1       196                         7                       9   
        10258                       7                       9   
        10326                       1                       1   
        12427                       7                       9   
        13032                       3                       3   
...                               ...                     ...   
206209  43961                       3                       3   
        44325                       1                       1   
        48370                       1                       1   
        48697                       1                       1   
        48742                       2                       2   

                    add_to_cart_order  reordered  
user_id product_id                                
1       196                         3          2  
        10258                       4          2  
        10326                       1          1  
        12427                       5          2  
        13032                       3          2  
...                               ...        ...  
206209  43961                       3          2  
        44325                       1          1  
        48370                       1          1  
        48697                       1          1  
        48742                       2          2  

[13307953 rows x 8 columns]

In [12]:
#up_uniqie_dow : user product no of unique with day of week.
up_unique_dow = prior_orders.groupby(["user_id","product_id"])["order_dow"].nunique().to_frame("up_unique_dow").reset_index()
up_unique_dow

,user_id,product_id,up_unique_dow
0,1,196,4
1,1,10258,4
2,1,10326,1
3,1,12427,4
4,1,13032,3
...,...,...,...
13307948,206209,43961,2
13307949,206209,44325,1
13307950,206209,48370,1
13307951,206209,48697,1


In [13]:
instances.append(up_unique_dow)

In [14]:
#up_unique_hod : user product no of unique with hour of the day
up_unique_hod = prior_orders.groupby(["user_id","product_id"])["order_hour_of_day"].nunique().to_frame("up_unique_hod").reset_index()
up_unique_hod

,user_id,product_id,up_unique_hod
0,1,196,7
1,1,10258,7
2,1,10326,1
3,1,12427,7
4,1,13032,3
...,...,...,...
13307948,206209,43961,3
13307949,206209,44325,1
13307950,206209,48370,1
13307951,206209,48697,1


In [15]:
instances.append(up_unique_hod)

In [16]:
#up_unique_atc : user product no of unique with add to cart order
up_unique_atc = prior_orders.groupby(["user_id","product_id"])["add_to_cart_order"].nunique().to_frame("up_unique_atc").reset_index()
up_unique_atc

,user_id,product_id,up_unique_atc
0,1,196,3
1,1,10258,4
2,1,10326,1
3,1,12427,5
4,1,13032,3
...,...,...,...
13307948,206209,43961,3
13307949,206209,44325,1
13307950,206209,48370,1
13307951,206209,48697,1


In [ ]:
instances.append(up_unique_atc)

In [18]:
#up_unique_reordered :   user product no of unique with reorder
up_unique_reorder = prior_orders.groupby(["user_id","product_id"])["reordered"].nunique().to_frame("up_unique_reordered").reset_index()
up_unique_reorder

,user_id,product_id,up_unique_reordered
0,1,196,2
1,1,10258,2
2,1,10326,1
3,1,12427,2
4,1,13032,2
...,...,...,...
13307948,206209,43961,2
13307949,206209,44325,1
13307950,206209,48370,1
13307951,206209,48697,1


In [19]:
instances.append(up_unique_reorder)

In [20]:
#up_cumcount_on : user product sum with order number
up_cumcount_on = prior_orders.groupby(["user_id","product_id"])["order_number"].sum().to_frame("up_cumcount_on").reset_index()
up_cumcount_on

,user_id,product_id,up_cumcount_on
0,1,196,55
1,1,10258,54
2,1,10326,5
3,1,12427,55
4,1,13032,19
...,...,...,...
13307948,206209,43961,24
13307949,206209,44325,7
13307950,206209,48370,11
13307951,206209,48697,7


In [21]:
instances.append(up_cumcount_on)

In [22]:
#up_cumcount_dow : user product sum with day of week
up_cumcount_dow = prior_orders.groupby(["user_id","product_id"])["order_dow"].sum().to_frame("up_cumcount_dow").reset_index()
up_cumcount_dow

,user_id,product_id,up_cumcount_dow
0,1,196,25
1,1,10258,23
2,1,10326,4
3,1,12427,25
4,1,13032,8
...,...,...,...
13307948,206209,43961,5
13307949,206209,44325,2
13307950,206209,48370,4
13307951,206209,48697,2


In [23]:
instances.append(up_cumcount_dow)

In [24]:
#up_cumcount_hod : user product sum with hour of day
up_cumcount_hod = prior_orders.groupby(["user_id","product_id"])["order_hour_of_day"].sum().to_frame("up_cumcount_hod").reset_index()
up_cumcount_hod

,user_id,product_id,up_cumcount_hod
0,1,196,103
1,1,10258,95
2,1,10326,15
3,1,12427,103
4,1,13032,24
...,...,...,...
13307948,206209,43961,39
13307949,206209,44325,13
13307950,206209,48370,10
13307951,206209,48697,13


In [25]:
instances.append(up_cumcount_hod)

In [26]:
#up_cumcount_dsp : user product sum with days since prior order
up_cumcount_dsp = prior_orders.groupby(["user_id","product_id"])["days_since_prior_order"].sum().to_frame("up_cumcount_dsp").reset_index()
up_cumcount_dsp

,user_id,product_id,up_cumcount_dsp
0,1,196,176.0
1,1,10258,176.0
2,1,10326,28.0
3,1,12427,176.0
4,1,13032,65.0
...,...,...,...
13307948,206209,43961,70.0
13307949,206209,44325,9.0
13307950,206209,48370,30.0
13307951,206209,48697,9.0


In [27]:
instances.append(up_cumcount_dsp)

In [28]:
#up_cumcount_atc : user product sum with add to cart order
up_cumcount_atc = prior_orders.groupby(["user_id","product_id"])["add_to_cart_order"].sum().to_frame("up_cumcount_atc").reset_index()
up_cumcount_atc

,user_id,product_id,up_cumcount_atc
0,1,196,14
1,1,10258,30
2,1,10326,5
3,1,12427,33
4,1,13032,19
...,...,...,...
13307948,206209,43961,24
13307949,206209,44325,8
13307950,206209,48370,8
13307951,206209,48697,6


In [29]:
instances.append(up_cumcount_atc)

In [30]:
# up_orders_cumcount : cumulative sum of prodect bough with days since previous ordered.
up_orders_cumcount =  prior_orders.groupby(["user_id","product_id"]).count()

up_orders_cumcount = up_orders_cumcount["days_since_prior_order"].to_frame("up_orders_cumcount").reset_index()
up_orders_cumcount

,user_id,product_id,up_orders_cumcount
0,1,196,9
1,1,10258,9
2,1,10326,1
3,1,12427,9
4,1,13032,3
...,...,...,...
13307948,206209,43961,3
13307949,206209,44325,1
13307950,206209,48370,1
13307951,206209,48697,1


In [31]:
instances.append(up_orders_cumcount)

In [32]:
#up_orders_num : Number of times user ordered the products
up_orders_num = prior_orders.groupby(["user_id","product_id"])["order_id"].count().to_frame("up_orders_num").reset_index()
up_orders_num

,user_id,product_id,up_orders_num
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3
...,...,...,...
13307948,206209,43961,3
13307949,206209,44325,1
13307950,206209,48370,1
13307951,206209,48697,1


In [33]:
instances.append(up_orders_num)

In [34]:
#up_orders_since_previous : user max order number - user-product max order number
user_number = prior_orders.groupby(['user_id'])['order_number'].max().to_frame("user_number").reset_index()
user_product_number = prior_orders.groupby(["user_id" , "product_id"])["order_number"].max().to_frame("user_product_number").reset_index()

df = pd.merge(user_product_number , user_number , on="user_id" , how = "left")
df["up_orders_since_previous"] = df["user_number"] - df["user_product_number"]
up_orders_since_previous = df[["user_id" , "product_id" ,"up_orders_since_previous"]]
up_orders_since_previous

,user_id,product_id,up_orders_since_previous
0,1,196,0
1,1,10258,0
2,1,10326,5
3,1,12427,0
4,1,13032,0
...,...,...,...
13307948,206209,43961,1
13307949,206209,44325,6
13307950,206209,48370,2
13307951,206209,48697,6


In [35]:
del user_number
del user_product_number
del df
instances.append(up_orders_since_previous)

In [36]:
#up_reordered : user_product reordered count
up_reordered = prior_orders.groupby(['user_id', 'product_id'])['reordered'].sum().to_frame("up_reordered").reset_index()
up_reordered

,user_id,product_id,up_reordered
0,1,196,9
1,1,10258,8
2,1,10326,0
3,1,12427,9
4,1,13032,2
...,...,...,...
13307948,206209,43961,2
13307949,206209,44325,0
13307950,206209,48370,0
13307951,206209,48697,0


In [37]:
instances.append(up_reordered)

In [38]:
#up_order_ratio :user-product order ratio
up_order_ratio_del = prior_orders.groupby(['user_id', 'product_id'])['order_number'].count().to_frame("up_order_ratio_del").reset_index()
user_order_ratio = prior_orders.groupby(by="user_id")["order_number"].max().to_frame("user_order_ratio").reset_index()

df = pd.merge(up_order_ratio_del , user_order_ratio , on="user_id" , how="left")
df["up_order_ratio"] = df["up_order_ratio_del"] / df["user_order_ratio"]
up_order_ratio = df[["user_id" , "product_id" , "up_order_ratio"]]
up_order_ratio

,user_id,product_id,up_order_ratio
0,1,196,1.000000
1,1,10258,0.900000
2,1,10326,0.100000
3,1,12427,1.000000
4,1,13032,0.300000
...,...,...,...
13307948,206209,43961,0.230769
13307949,206209,44325,0.076923
13307950,206209,48370,0.076923
13307951,206209,48697,0.076923


In [39]:
del up_order_ratio_del
del user_order_ratio
del df
instances.append(up_order_ratio)

In [40]:
#up_order_dow_score :user product order day of the week mean
up_order_dow_score = prior_orders.groupby(["user_id" , "product_id"])["order_dow"].mean().to_frame("up_order_dow_score").reset_index()
up_order_dow_score

,user_id,product_id,up_order_dow_score
0,1,196,2.500000
1,1,10258,2.555556
2,1,10326,4.000000
3,1,12427,2.500000
4,1,13032,2.666667
...,...,...,...
13307948,206209,43961,1.666667
13307949,206209,44325,2.000000
13307950,206209,48370,4.000000
13307951,206209,48697,2.000000


In [41]:
instances.append(up_order_dow_score)

In [42]:
#up_order_hour_of_day_score : user product ordere hour of the day mean
up_order_hour_of_day_score = prior_orders.groupby(["user_id" , "product_id"])["order_hour_of_day"].mean().to_frame("up_order_hour_of_day_score").reset_index()
up_order_hour_of_day_score

,user_id,product_id,up_order_hour_of_day_score
0,1,196,10.300000
1,1,10258,10.555556
2,1,10326,15.000000
3,1,12427,10.300000
4,1,13032,8.000000
...,...,...,...
13307948,206209,43961,13.000000
13307949,206209,44325,13.000000
13307950,206209,48370,10.000000
13307951,206209,48697,13.000000


In [43]:
instances.append(up_order_hour_of_day_score)

In [44]:
#up_last_five : times a customer bought a product on its last 5 orders
#URL : https://www.kaggle.com/errolpereira/xgboost-with-feature-engineering(Applied AI Team Recommended)
prior_orders['order_number_back'] = prior_orders.groupby(by=['user_id'])['order_number'].transform(max) - prior_orders.order_number + 1
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
0,2539329,1,prior,1,2,8,NaN,196,1,0,10
1,2539329,1,prior,1,2,8,NaN,14084,2,0,10
2,2539329,1,prior,1,2,8,NaN,12427,3,0,10
3,2539329,1,prior,1,2,8,NaN,26088,4,0,10
4,2539329,1,prior,1,2,8,NaN,26405,5,0,10


In [45]:
temp = prior_orders.loc[prior_orders.order_number_back <= 5]
temp.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
29,3367565,1,prior,6,2,7,19.0,196,1,1,5
30,3367565,1,prior,6,2,7,19.0,12427,2,1,5
31,3367565,1,prior,6,2,7,19.0,10258,3,1,5
32,3367565,1,prior,6,2,7,19.0,25133,4,1,5
33,550135,1,prior,7,1,9,20.0,196,1,1,4


In [46]:
up_last_five = temp.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').to_frame('up_last_five').reset_index()
up_last_five

,user_id,product_id,up_last_five
0,1,196,5
1,1,10258,5
2,1,12427,5
3,1,13032,2
4,1,25133,5
...,...,...,...
6714240,206209,40310,1
6714241,206209,41213,3
6714242,206209,43961,1
6714243,206209,48370,1


In [47]:
#up_last_five_ratio
up_last_five["up_last_five_ratio"] = up_last_five["up_last_five"] / 5
up_last_five

,user_id,product_id,up_last_five,up_last_five_ratio
0,1,196,5,1.0
1,1,10258,5,1.0
2,1,12427,5,1.0
3,1,13032,2,0.4
4,1,25133,5,1.0
...,...,...,...,...
6714240,206209,40310,1,0.2
6714241,206209,41213,3,0.6
6714242,206209,43961,1,0.2
6714243,206209,48370,1,0.2


In [48]:
instances.append(up_last_five)

In [49]:
len(instances)

17

<h1> Concatenating all user-product features </h1>

In [50]:
user_product = pd.DataFrame()
user_product["user_id"] = up_order_hour_of_day_score.user_id.copy()
user_product["product_id"] = up_order_hour_of_day_score.product_id.copy()

print("Merging all the user-product features..")
for instance in tqdm(instances):
    user_product = pd.merge(user_product , instance , on = ['user_id', 'product_id'], how='left')
print("Merging Done...")

print(user_product.info())

  0%|          | 0/17 [00:00<?, ?it/s]

Merging all the user-product features..


100%|██████████| 17/17 [02:05<00:00,  7.38s/it]

Merging Done...
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 20 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   user_id                     int64  
 1   product_id                  int64  
 2   up_unique_dow               int64  
 3   up_unique_hod               int64  
 4   up_unique_atc               int64  
 5   up_unique_reordered         int64  
 6   up_cumcount_on              int64  
 7   up_cumcount_dow             int64  
 8   up_cumcount_hod             int64  
 9   up_cumcount_dsp             float64
 10  up_cumcount_atc             int64  
 11  up_orders_cumcount          int64  
 12  up_orders_num               int64  
 13  up_orders_since_previous    int64  
 14  up_reordered                int64  
 15  up_order_ratio              float64
 16  up_order_dow_score          float64
 17  up_order_hour_of_day_score  float64
 18  up_last_five                float64
 19  up_

In [51]:
user_product.fillna(0 , inplace = True)

user_product = reduce_mem_usage(user_product)

with open("user_product.pickle" , "wb") as file:
    pickle.dump(user_product , file)


Memory usage after optimization is: 558.42 MB
Decreased by 73.8%


In [52]:
#Cleaning
print("Cleaning....")
for instance in instances:
    del instance
print("Done")

Cleaning....
Done


In [53]:
del user_product

<h1> Concatenating All the Features </h1>

In [39]:
#Discarding Cyclic Features as already taken the behaviour with user and products
#cyclic_features = pickle.load(open('cyclic_features.pickle' , "rb"))
user = pickle.load(open("users.pickle" , 'rb'))
products = pickle.load(open("products.pickle" , "rb"))
user_products = pickle.load(open("user_product.pickle" , "rb"))


data = pd.merge(user_products , user , on="user_id" , how="left")
data = pd.merge(data , products , on="product_id" , how="left")
#data = pd.merge(data , cyclic_features , on = ["user_id" , "product_id"] , how='left')
data

,user_id,product_id,up_unique_dow,up_unique_hod,up_unique_atc,up_unique_reordered,up_cumcount_on,up_cumcount_dow,up_cumcount_hod,up_cumcount_dsp,...,prod_sum_dsp,prod_sum_atc,prod_sum_reordered,prod_num,prod_unique_users,prod_reorder_mean,prod_order_hour_of_day_mean,prod_order_hour_of_day_var,prod_order_dow_mean,prod_order_dow_var
0,1,196,4,7,3,2,55,25,103,176.0,...,400709.0,133206,27791,35791,8000,0.776367,12.523438,13.890625,2.898438,3.115234
1,1,10258,4,7,4,2,54,23,95,176.0,...,21600.0,8324,1389,1946,557,0.713867,12.257812,12.476562,2.718750,2.888672
2,1,10326,1,1,1,1,5,4,15,28.0,...,61768.0,23160,3603,5526,1923,0.651855,12.898438,18.203125,2.742188,3.822266
3,1,12427,4,7,5,2,55,25,103,176.0,...,64474.0,30826,4797,6476,1679,0.740723,12.265625,12.414062,2.669922,2.857422
4,1,13032,3,3,3,2,19,8,24,65.0,...,39823.0,21091,2465,3751,1286,0.657227,12.578125,16.187500,2.779297,3.355469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13307948,206209,43961,2,3,3,2,24,5,39,70.0,...,573302.0,509121,34916,55371,20455,0.630371,13.507812,18.187500,2.587891,4.535156
13307949,206209,44325,1,1,1,1,7,2,13,9.0,...,40318.0,35233,1398,3485,2087,0.401123,13.421875,18.671875,2.818359,4.347656
13307950,206209,48370,1,1,1,1,11,4,10,30.0,...,40844.0,32829,2751,3934,1183,0.699219,13.250000,16.718750,2.779297,4.210938
13307951,206209,48697,1,1,1,1,7,2,13,9.0,...,102043.0,85729,3499,9783,6284,0.357666,13.328125,19.125000,2.859375,4.464844


In [40]:
del user
del products
del user_products

In [41]:
data.fillna(0,inplace = True)

In [42]:
with open("data.pickle" , "wb") as file:
    pickle.dump(data,file)

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 49 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   user_id                      int32  
 1   product_id                   int32  
 2   up_unique_dow                int8   
 3   up_unique_hod                int8   
 4   up_unique_atc                int8   
 5   up_unique_reordered          int8   
 6   up_cumcount_on               int16  
 7   up_cumcount_dow              int16  
 8   up_cumcount_hod              int16  
 9   up_cumcount_dsp              float16
 10  up_cumcount_atc              int16  
 11  up_orders_cumcount           int8   
 12  up_orders_num                int8   
 13  up_orders_since_previous     int8   
 14  up_reordered                 int8   
 15  up_order_ratio               float16
 16  up_order_dow_score           float16
 17  up_order_hour_of_day_score   float16
 18  up_last_five                 float16
 19

<h1> Creating Train and Test Data set </h1>

In [3]:
data = pickle.load(open("data.pickle","rb"))

In [44]:
data = reduce_mem_usage(data)

Memory usage after optimization is: 1484.90 MB
Decreased by 0.0%


In [45]:
orders_filtered = orders_data[(orders_data.eval_set == "train") | (orders_data.eval_set == "test")]

In [46]:
#orders_filtered = orders_data[(orders_data.eval_set == "train") | (orders_data.eval_set == "test")]
orders_filtered = orders_filtered[['user_id', 'eval_set', 'order_id']]
orders_filtered

,user_id,eval_set,order_id
10,1,train,1187899
25,2,train,1492625
38,3,test,2774568
44,4,test,329954
49,5,train,2196797
...,...,...,...
3420933,206205,train,1716008
3421001,206206,test,1043943
3421018,206207,test,2821651
3421068,206208,test,803273


In [47]:
data = pd.merge(data , orders_filtered , on="user_id" , how="left")
data

,user_id,product_id,up_unique_dow,up_unique_hod,up_unique_atc,up_unique_reordered,up_cumcount_on,up_cumcount_dow,up_cumcount_hod,up_cumcount_dsp,...,prod_sum_reordered,prod_num,prod_unique_users,prod_reorder_mean,prod_order_hour_of_day_mean,prod_order_hour_of_day_var,prod_order_dow_mean,prod_order_dow_var,eval_set,order_id
0,1,196,4,7,3,2,55,25,103,176.0,...,27791,35791,8000,0.776367,12.523438,13.890625,2.898438,3.115234,train,1187899
1,1,10258,4,7,4,2,54,23,95,176.0,...,1389,1946,557,0.713867,12.257812,12.476562,2.718750,2.888672,train,1187899
2,1,10326,1,1,1,1,5,4,15,28.0,...,3603,5526,1923,0.651855,12.898438,18.203125,2.742188,3.822266,train,1187899
3,1,12427,4,7,5,2,55,25,103,176.0,...,4797,6476,1679,0.740723,12.265625,12.414062,2.669922,2.857422,train,1187899
4,1,13032,3,3,3,2,19,8,24,65.0,...,2465,3751,1286,0.657227,12.578125,16.187500,2.779297,3.355469,train,1187899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13307948,206209,43961,2,3,3,2,24,5,39,70.0,...,34916,55371,20455,0.630371,13.507812,18.187500,2.587891,4.535156,train,272231
13307949,206209,44325,1,1,1,1,7,2,13,9.0,...,1398,3485,2087,0.401123,13.421875,18.671875,2.818359,4.347656,train,272231
13307950,206209,48370,1,1,1,1,11,4,10,30.0,...,2751,3934,1183,0.699219,13.250000,16.718750,2.779297,4.210938,train,272231
13307951,206209,48697,1,1,1,1,7,2,13,9.0,...,3499,9783,6284,0.357666,13.328125,19.125000,2.859375,4.464844,train,272231


In [48]:
data.fillna(0,inplace = True)

In [49]:
#Training Dataset
data_train = data[data.eval_set == "train"]
data_train

,user_id,product_id,up_unique_dow,up_unique_hod,up_unique_atc,up_unique_reordered,up_cumcount_on,up_cumcount_dow,up_cumcount_hod,up_cumcount_dsp,...,prod_sum_reordered,prod_num,prod_unique_users,prod_reorder_mean,prod_order_hour_of_day_mean,prod_order_hour_of_day_var,prod_order_dow_mean,prod_order_dow_var,eval_set,order_id
0,1,196,4,7,3,2,55,25,103,176.0,...,27791,35791,8000,0.776367,12.523438,13.890625,2.898438,3.115234,train,1187899
1,1,10258,4,7,4,2,54,23,95,176.0,...,1389,1946,557,0.713867,12.257812,12.476562,2.718750,2.888672,train,1187899
2,1,10326,1,1,1,1,5,4,15,28.0,...,3603,5526,1923,0.651855,12.898438,18.203125,2.742188,3.822266,train,1187899
3,1,12427,4,7,5,2,55,25,103,176.0,...,4797,6476,1679,0.740723,12.265625,12.414062,2.669922,2.857422,train,1187899
4,1,13032,3,3,3,2,19,8,24,65.0,...,2465,3751,1286,0.657227,12.578125,16.187500,2.779297,3.355469,train,1187899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13307948,206209,43961,2,3,3,2,24,5,39,70.0,...,34916,55371,20455,0.630371,13.507812,18.187500,2.587891,4.535156,train,272231
13307949,206209,44325,1,1,1,1,7,2,13,9.0,...,1398,3485,2087,0.401123,13.421875,18.671875,2.818359,4.347656,train,272231
13307950,206209,48370,1,1,1,1,11,4,10,30.0,...,2751,3934,1183,0.699219,13.250000,16.718750,2.779297,4.210938,train,272231
13307951,206209,48697,1,1,1,1,7,2,13,9.0,...,3499,9783,6284,0.357666,13.328125,19.125000,2.859375,4.464844,train,272231


In [50]:
#Getting the information for the Trainig data from train data
data_train = data_train.merge(or_prod_train[['product_id', 'order_id', 'reordered']], on=['product_id', 'order_id'], how='left')
data_train

,user_id,product_id,up_unique_dow,up_unique_hod,up_unique_atc,up_unique_reordered,up_cumcount_on,up_cumcount_dow,up_cumcount_hod,up_cumcount_dsp,...,prod_num,prod_unique_users,prod_reorder_mean,prod_order_hour_of_day_mean,prod_order_hour_of_day_var,prod_order_dow_mean,prod_order_dow_var,eval_set,order_id,reordered
0,1,196,4,7,3,2,55,25,103,176.0,...,35791,8000,0.776367,12.523438,13.890625,2.898438,3.115234,train,1187899,1.0
1,1,10258,4,7,4,2,54,23,95,176.0,...,1946,557,0.713867,12.257812,12.476562,2.718750,2.888672,train,1187899,1.0
2,1,10326,1,1,1,1,5,4,15,28.0,...,5526,1923,0.651855,12.898438,18.203125,2.742188,3.822266,train,1187899,NaN
3,1,12427,4,7,5,2,55,25,103,176.0,...,6476,1679,0.740723,12.265625,12.414062,2.669922,2.857422,train,1187899,NaN
4,1,13032,3,3,3,2,19,8,24,65.0,...,3751,1286,0.657227,12.578125,16.187500,2.779297,3.355469,train,1187899,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8474656,206209,43961,2,3,3,2,24,5,39,70.0,...,55371,20455,0.630371,13.507812,18.187500,2.587891,4.535156,train,272231,NaN
8474657,206209,44325,1,1,1,1,7,2,13,9.0,...,3485,2087,0.401123,13.421875,18.671875,2.818359,4.347656,train,272231,NaN
8474658,206209,48370,1,1,1,1,11,4,10,30.0,...,3934,1183,0.699219,13.250000,16.718750,2.779297,4.210938,train,272231,NaN
8474659,206209,48697,1,1,1,1,7,2,13,9.0,...,9783,6284,0.357666,13.328125,19.125000,2.859375,4.464844,train,272231,NaN


In [51]:
#filling the NAN values in the reordered
data_train.reordered.fillna(0, inplace=True)

#setting user_id and product_id as index.
data_train = data_train.set_index(['user_id', 'product_id'])

#deleting eval_set, order_id as they are not needed for training.
data_train.drop(['eval_set', 'order_id'], axis=1, inplace=True)

#head()
data_train.head()

up_unique_dow  up_unique_hod  up_unique_atc  \
user_id product_id                                                
1       196                     4              7              3   
        10258                   4              7              4   
        10326                   1              1              1   
        12427                   4              7              5   
        13032                   3              3              3   

                    up_unique_reordered  up_cumcount_on  up_cumcount_dow  \
user_id product_id                                                         
1       196                           2              55               25   
        10258                         2              54               23   
        10326                         1               5                4   
        12427                         2              55               25   
        13032                         2              19                8   

                    up_cumcount_hod  up_cumcount_dsp  up_cumcount_atc  \
user_id product_id                                                      
1       196                     103            176.0               14   
        10258                    95            176.0               30   
        10326                    15             28.0                5   
        12427                   103            176.0               33   
        13032                    24             65.0               19   

                    up_orders_cumcount  ...  prod_sum_atc  prod_sum_reordered  \
user_id product_id                      ...                                     
1       196                          9  ...        133206               27791   
        10258                        9  ...          8324                1389   
        10326                        1  ...         23160                3603   
        12427                        9  ...         30826                4797   
        13032                        3  ...         21091                2465   

                    prod_num  prod_unique_users  prod_reorder_mean  \
user_id product_id                                                   
1       196            35791               8000           0.776367   
        10258           1946                557           0.713867   
        10326           5526               1923           0.651855   
        12427           6476               1679           0.740723   
        13032           3751               1286           0.657227   

                    prod_order_hour_of_day_mean  prod_order_hour_of_day_var  \
user_id product_id                                                            
1       196                           12.523438                   13.890625   
        10258                         12.257812                   12.476562   
        10326                         12.898438                   18.203125   
        12427                         12.265625                   12.414062   
        13032                         12.578125                   16.187500   

                    prod_order_dow_mean  prod_order_dow_var  reordered  
user_id product_id                                                      
1       196                    2.898438            3.115234        1.0  
        10258                  2.718750            2.888672        1.0  
        10326                  2.742188            3.822266        0.0  
        12427                  2.669922            2.857422        0.0  
        13032                  2.779297            3.355469        1.0  

[5 rows x 48 columns]

In [52]:
#Creating Test Dataset
data_test = data[data.eval_set == "test"]
data_test

,user_id,product_id,up_unique_dow,up_unique_hod,up_unique_atc,up_unique_reordered,up_cumcount_on,up_cumcount_dow,up_cumcount_hod,up_cumcount_dsp,...,prod_sum_reordered,prod_num,prod_unique_users,prod_reorder_mean,prod_order_hour_of_day_mean,prod_order_hour_of_day_var,prod_order_dow_mean,prod_order_dow_var,eval_set,order_id
120,3,248,1,1,1,1,2,3,19,9.0,...,2550,6371,3821,0.400146,13.546875,19.109375,2.662109,4.570312,test,2774568
121,3,1005,1,1,1,1,10,3,16,17.0,...,204,463,259,0.440674,12.695312,14.375000,2.853516,3.134766,test,2774568
122,3,1819,2,3,3,2,17,2,49,34.0,...,1193,2424,1231,0.492188,13.429688,17.437500,2.687500,4.226562,test,2774568
123,3,7503,1,1,1,1,3,3,16,21.0,...,6905,12474,5569,0.553711,13.390625,18.171875,2.662109,4.292969,test,2774568
124,3,8021,1,1,1,1,2,3,19,9.0,...,16472,27864,11392,0.591309,13.476562,18.562500,2.775391,4.425781,test,2774568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13307880,206208,48364,1,1,1,1,3,2,15,7.0,...,10338,19501,9163,0.530273,13.468750,20.437500,2.843750,4.464844,test,803273
13307881,206208,48865,1,1,1,1,42,1,19,6.0,...,316,802,486,0.394043,13.351562,17.421875,2.474609,4.777344,test,803273
13307882,206208,49247,1,1,1,1,6,1,18,7.0,...,4119,6292,2173,0.654785,13.398438,19.328125,2.753906,4.371094,test,803273
13307883,206208,49385,1,1,1,1,27,2,14,7.0,...,57,318,261,0.179199,13.296875,16.375000,2.996094,4.906250,test,803273


In [53]:
#setting user_id and product_id as index.
data_test = data_test.set_index(['user_id', 'product_id'])

#deleting eval_set, order_id as they are not needed for training.
data_test.drop(['eval_set', 'order_id'], axis=1, inplace=True)

data_test.head()

up_unique_dow  up_unique_hod  up_unique_atc  \
user_id product_id                                                
3       248                     1              1              1   
        1005                    1              1              1   
        1819                    2              3              3   
        7503                    1              1              1   
        8021                    1              1              1   

                    up_unique_reordered  up_cumcount_on  up_cumcount_dow  \
user_id product_id                                                         
3       248                           1               2                3   
        1005                          1              10                3   
        1819                          2              17                2   
        7503                          1               3                3   
        8021                          1               2                3   

                    up_cumcount_hod  up_cumcount_dsp  up_cumcount_atc  \
user_id product_id                                                      
3       248                      19              9.0                3   
        1005                     16             17.0                5   
        1819                     49             34.0                8   
        7503                     16             21.0                6   
        8021                     19              9.0                5   

                    up_orders_cumcount  ...  prod_sum_dsp  prod_sum_atc  \
user_id product_id                      ...                               
3       248                          1  ...       65657.0         67665   
        1005                         1  ...        4514.0          4398   
        1819                         3  ...       24038.0         22513   
        7503                         1  ...      128062.0        119094   
        8021                         1  ...      285674.0        245840   

                    prod_sum_reordered  prod_num  prod_unique_users  \
user_id product_id                                                    
3       248                       2550      6371               3821   
        1005                       204       463                259   
        1819                      1193      2424               1231   
        7503                      6905     12474               5569   
        8021                     16472     27864              11392   

                    prod_reorder_mean  prod_order_hour_of_day_mean  \
user_id product_id                                                   
3       248                  0.400146                    13.546875   
        1005                 0.440674                    12.695312   
        1819                 0.492188                    13.429688   
        7503                 0.553711                    13.390625   
        8021                 0.591309                    13.476562   

                    prod_order_hour_of_day_var  prod_order_dow_mean  \
user_id product_id                                                    
3       248                          19.109375             2.662109   
        1005                         14.375000             2.853516   
        1819                         17.437500             2.687500   
        7503                         18.171875             2.662109   
        8021                         18.562500             2.775391   

                    prod_order_dow_var  
user_id product_id                      
3       248                   4.570312  
        1005                  3.134766  
        1819                  4.226562  
        7503                  4.292969  
        8021                  4.425781  

[5 rows x 47 columns]

In [54]:
print(data_train.shape , data_test.shape)

(8474661, 48) (4833292, 47)


In [55]:
#Saving the Train and test files
data_train = reduce_mem_usage(data_train)
data_test = reduce_mem_usage(data_test)

print("Saving File....")
with open("data_train.pickle" , "wb") as file:
    pickle.dump(data_train , file)
with open("data_test.pickle" , "wb") as file:
    pickle.dump(data_test , file)
print("Done..")


Memory usage after optimization is: 904.74 MB
Decreased by 5.1%
Memory usage after optimization is: 507.12 MB
Decreased by 0.0%
Saving File....
Done..
